This notebook demonstrates how ebweyl can be used to compute the electric and magnetic parts of the Weyl tensor of a given spacetime and classify it according to the Petrov types.

In this process we will compute the spatial Ricci tensor and scalar as well as the Weyl scalar for an arbitrary tetrad base and the invariant scalars: I, J, K, L , N.

In [1]:
import numpy as np
import ebweyl as ebw

The documentation on the ebweyl module is accessible with the help() function.

In [2]:
help(ebw.Weyl.ricci_tensor_down3)

Help on function ricci_tensor_down3 in module ebweyl:

ricci_tensor_down3(self, Gudd3)
    Compute spatial Ricci tensor with both indices down.
    
    Parameters : 
        Gudd3 : (3, 3, 3, N, N, N) array_like 
                Spatial Christoffel symbol with one indice up and two down
                You can get this from Weyl.christoffel_symbol_udd3()
            
    Returns : 
        (3, 3, N, N, N) array_like



# Define the Spacetime

For this example we use the A.Harvey and T.Tsoubelis vacuum Bianchi IV plane wave homogeneous spacetime (see page 191 of 'Dynamical Systems in Cosmology' by J.Wainwright and G.F.R.Ellis). With Cartesian coordinates the metric takes the form:

$$
g_{\alpha\beta} = 
\begin{pmatrix}
   -1 &  0  & 0 & 0 \\
    0 & t^2 & 0 & 0 \\
    0 &  0  & te^{x} & te^{x}(x+\log{t}) \\
    0 &  0  & te^{x}(x+\log{t}) & te^{x}((x+\log{t})^2+1)
\end{pmatrix}
$$

This is an analytical spacetime but the code presented here works for any numerical spacetime, the user needs the metric, extrinsic curvature and stress-energy tensor. 

For this metric the extrinsic curvature is: $K_{\alpha\beta} = \frac{-1}{2}\partial_{t}(g_{\alpha\beta})$.

This metric is a solution to vacuum, so the stress-energy tensor is: $T_{\alpha\beta} = 0$.

Let's create arrays for these tensors. 

#### Create a data grid

In [3]:
L = 20  # Data box size
# If you are running this on your own laptop and don't want the 
# computation to take too much time, you should make N smaller. 
# (But keep in mind that the accuracy increases as N increases)
N = 128  # Number of data points per side
dx = L / N  # Elementary grid size

# Cartesian coordinates
xyz = np.arange(- L / 2, L / 2, dx)
x, y, z = np.meshgrid(xyz, xyz, xyz, indexing='ij')

#### Create metric tensor array

In [4]:
t = 1.5  # An arbitrary time
B = (x+np.log(t))  # Function from referenced book
Box_0 = np.zeros([N, N, N])
Box_1 = np.ones([N, N, N])

gdown4 = np.array([[-Box_1, Box_0, Box_0, Box_0],
                   [Box_0, (t*t)*Box_1, Box_0, Box_0],
                   [Box_0, Box_0, t*np.exp(x), t*np.exp(x)*B],
                   [Box_0, Box_0, t*np.exp(x)*B, 
                    t*np.exp(x)*(B*B+1)]])

#### Create extrinsic curvature tensor array

In [5]:
dtB = 1/t  # Time derivative of B function
# Time derivative of metric
dtgdown4 = np.array([[Box_0, Box_0, Box_0, Box_0],
                     [Box_0, 2*t*Box_1, Box_0, Box_0],
                     [Box_0, Box_0, np.exp(x), 
                      np.exp(x)*B + t*np.exp(x)*dtB],
                     [Box_0, Box_0, np.exp(x)*B + t*np.exp(x)*dtB, 
                      np.exp(x)*(B*B+1) + t*np.exp(x)*(2*dtB*B)]])

Kdown4 = (-1/2)*dtgdown4

#### Create stress-energy tensor array

In [6]:
kappa = 8*np.pi  # Einstein's constant with G = c = 1
Tdown4 = np.zeros([4, 4, N, N, N])

# Electric $E_{\alpha\beta}$ and magnetic $B_{\alpha\beta}$ parts of the Weyl tensor

#### Define the FiniteDifference class

Default values are : periodic_boundary=True, fd_order6=False

But periodic boundaries would be inapropriate for this spacetime, the code will then use a combination of forward, centered and backward finite difference schemes instead.

In [7]:
FD = ebw.FiniteDifference(dx, N, periodic_boundary=False, 
                          fd_order6=True)

#### Define the Weyl class

The FD class is given to the Weyl class and we won't use it again here, but you can use it on it's own to compute spatial derives.

In [8]:
EBW = ebw.Weyl(FD, gdown4, Kdown4)

This will automatically compute the standard terms of the 3+1 formulation, for example:

In [9]:
print('lapse = ', np.mean(EBW.alpha))

lapse =  1.0


Notice that here I'm showing the mean. This is because np.shape(EBW.alpha) = (N, N, N) as the lapse can depend on space.

#### Compute Spatial Ricci tensor

In [10]:
Gudd3 = EBW.christoffel_symbol_udd3()
RicciTdown3 = EBW.ricci_tensor_down3(Gudd3)

This then easily provides the spatial Ricci scalar:

In [11]:
RicciS3 = EBW.trace_rank2tensor3(RicciTdown3)
print('R3 = ', np.mean(FD.cutoffmask(RicciS3)))
print('Analytical answer: ', -2/t**2)

R3 =  -0.888892287646
Analytical answer:  -0.8888888888888888


Notice that here I'm applying the FD.cutoffmask function before taking the mean.

In [12]:
help(FD.cutoffmask)

Help on method cutoffmask in module ebweyl:

cutoffmask(f) method of ebweyl.FiniteDifference instance
    Remove points affected by the boundary condition.



The backward and forward finite difference schemes slightly underperform compared to the centered one, so that is why I remove those before taking the mean. I wouldn't bother doing this if I were using periodic boundary conditions.

#### Compute $E_{\alpha\beta}$ and $B_{\alpha\beta}$ projected along the normal to the hypersurface

In [13]:
Endown3 = EBW.eweyl_n_tensor_down3(RicciTdown3, kappa, Tdown4)
Bndown3 = EBW.bweyl_n_tensor_down3(Gudd3)

These are both covariant tensors with both indices being spatial, hence the 3 at the end. 

If I want the indices to include time as well, I can do:

In [14]:
Endown4 = EBW.ebweyl_n_3D_to_4D(Endown3)

Lets look at their norm : $|E| = \sqrt{E_{\alpha\beta}E^{\alpha\beta}} = \sqrt{E_{ij}E^{ij}}$. Here Greek indices are spacetime and Latin indices are only space.

As this spacetime has a plane wave we have $|E|=|B|$.

In [15]:
En_norm = EBW.norm_rank2tensor3(Endown3)
# or = EBW.norm_rank2tensor4(Endown4)
Bn_norm = EBW.norm_rank2tensor3(Bndown3)

In [16]:
print('E^2 = ', np.mean(FD.cutoffmask(En_norm)))
print('B^2 = ', np.mean(FD.cutoffmask(Bn_norm)))
print('Analytical answer: ', 1/(np.sqrt(2)*t**2))

E^2 =  0.31427083223463154
B^2 =  0.31426956479598156
Analytical answer:  0.31426968052735443


### $E_{\alpha\beta}$ and $B_{\alpha\beta}$ along a different vector

With the terms projected along the normal to the hypersurface we can construct the Weyl tensor:

In [17]:
Cdown4 = EBW.weyl_tensor_down4(Endown3, Bndown3)

This can then be projected along any time-like vector. We chose the 4-velocity, for this metric it simply corresponds to the normal to the hypersurface:

In [18]:
uup4 = EBW.nup4  # spacetime indices up

Then the new $E_{\alpha\beta}$ and $B_{\alpha\beta}$ are obtained as:

In [19]:
Eudown4 = EBW.eweyl_u_tensor_down4(Cdown4, uup4)
Budown4 = EBW.bweyl_u_tensor_down4(Cdown4, uup4)

Because the 4-velocity here corresponds to the normal to the hypersurface, I demonstrate here that $|E|$ and $|B|$ are the same.

In [20]:
Eu_norm = EBW.norm_rank2tensor4(Eudown4)
Bu_norm = EBW.norm_rank2tensor4(Budown4)
print('|E| = ', np.mean(FD.cutoffmask(Eu_norm)))
print('|B| = ', np.mean(FD.cutoffmask(Bu_norm)))
print('Analytical answer: ', 1/(np.sqrt(2)*t**2))

|E| =  0.31427083223463154
|B| =  0.3142695647959815
Analytical answer:  0.31426968052735443


# Petrov type

To find the Petrov type, I construct the Weyl scalars $\Psi$ and then compute the scalar invariants, (J, I, L, K, N). Those are then used to determine the Petrov type according to Figure 9.1 of "Exact Solutions to Einstein's Field Equations" 2003, 2nd edition, by H.Stephani, D.Kramer, M.MacCallum, C.Hoenselaers and E.Herlt.

In order to compute the $\Psi$s I need the full Weyl tensor provided in cell [17].

In [21]:
Psis = EBW.weyl_psi_scalars(Cdown4)
print('Psi_0 = ', np.mean(FD.cutoffmask(np.real(Psis[0]))), ' + i(', np.mean(FD.cutoffmask(np.imag(Psis[0]))), ')')
print('Psi_1 = ', np.mean(FD.cutoffmask(np.real(Psis[1]))), ' + i(', np.mean(FD.cutoffmask(np.imag(Psis[1]))), ')')
print('Psi_2 = ', np.mean(FD.cutoffmask(np.real(Psis[2]))), ' + i(', np.mean(FD.cutoffmask(np.imag(Psis[2]))), ')')
print('Psi_3 = ', np.mean(FD.cutoffmask(np.real(Psis[3]))), ' + i(', np.mean(FD.cutoffmask(np.imag(Psis[3]))), ')')
print('Psi_4 = ', np.mean(FD.cutoffmask(np.real(Psis[4]))), ' + i(', np.mean(FD.cutoffmask(np.imag(Psis[4]))), ')')

Psi_0 =  8.962147990852077e-07  + i( 1.1107500664841863e-08 )
Psi_1 =  3.425016056026886e-15  + i( -3.235611977639702e-14 )
Psi_2 =  -1.0845286715418547e-07  + i( 2.445126914211757e-07 )
Psi_3 =  3.425016056026886e-15  + i( -3.235611977639702e-14 )
Psi_4 =  8.962147990852077e-07  + i( 1.1107500664841863e-08 )


These scalars are computed for an arbitrary null vector base, in order to use these for the Petrov classification we would need a frame rotation for these to be along the principal null directions. Instead we use the scalar invariants:

In [22]:
invars = EBW.invariant_scalars(Psis)

If $I^3 - 27 J^2 = 0$ it means that this spacetime is special, either of type II, D, III, N, or O,

otherwise it is of type I, the most general type.

In [23]:
val = invars['I']**3 - 27*invars['J']**2
np.mean(FD.cutoffmask(val**(1/6)).real)

1.0515536061089994e-06

Then it is a special type. 
Note: 
- here I put the value to the power of 1/6, that is to show a value of the same order of magnitude as the Weyl tensor. 
- here we are only considering the real part, to do this properly the imaginary part also should be considered.
- again I'm showing the mean, that is because this spacetime is homogeneous.
- this value will not be exactly zero, most importantly it will tend towards zero as we increase the resolution. One also would need to determine the numerical error with 2 other results obtained with different resolutions.

If $I = J = 0$ then the spacetime is either of type III, N, or O, otherwise it is of type II or D.

In [24]:
print('I^{1/2} = ', np.mean(FD.cutoffmask(invars['I']**(1/2)).real))
print('J^{1/3} = ', np.mean(FD.cutoffmask(invars['J']**(1/3)).real))

I^{1/2} =  8.162565314220748e-07
J^{1/3} =  4.710124326770538e-07


Then it is of type III, N, or O.

(Note, if it was of type II or D we would check $K$ and $N$ to distinguish them: if $K = N = 0$ then the spacetime is of type D, otherwise it is on type II.)

Next if $K = L = 0$ then the spacetime is of type N or O, otherwise it is on type III.

In [25]:
print('K^{1/3} = ', np.mean(FD.cutoffmask(invars['K']**(1/3)).real))
print('L^{1/2} = ', np.mean(FD.cutoffmask(invars['L']**(1/2)).real))
print('N^{1/4} = ', np.mean(FD.cutoffmask(invars['N']**(1/4)).real))

K^{1/3} =  4.796206140814408e-09
L^{1/2} =  2.644143196905683e-07
N^{1/4} =  7.885218033856684e-07


Then it is of type N, or O.

Next if $|B| = |E| = 0$ then the spacetime is of type O, it is conformally flat, otherwise it is of type N.

In [26]:
print('|B| = ', np.mean(FD.cutoffmask(Bu_norm)))
print('|E| = ', np.mean(FD.cutoffmask(Eu_norm)))

|B| =  0.3142695647959815
|E| =  0.31427083223463154


As we have shown previously $|B| = |E| \neq 0$ therefore this spacetime is of Petrov type N.